In [1]:
import pandas as pd
from plyer import notification
import requests
import os
import sqlite3
from contextlib import contextmanager
from datetime import datetime


In [2]:
token = "2528|6LQUYU9ZOJdvF24GGo4hTMKKCghaM6pk5yYtCarj"
headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {token}"
}
url = "https://restfulcountries.com/api/v1/countries"
response = requests.get(url, headers=headers)
data1 = None
if response.status_code == 200:
    data = response.json()
    data1 = data
else:
    print(f"Erro na requisição: {response.status_code}")
    print(response.text)

print('Funciona!')

Funciona!


In [3]:
def alerta_api(nivel, url, token, erro=""):
    '''
        Alerta sobre problemas com a API.
        Nível 1: Problema com o Token
        Nível 2: Erro na chamada da API (status code diferente de 200)
        Nível 3: API Indisponível (erro de conexão)
    '''
    now = str(datetime.now())
    title = ""
    msg = ""

    if nivel == 1:
        title = 'ATENÇÃO: Problema com o Token da API'
        msg = f"Possível problema com o token da API {url}.\n{now}\n{erro}"
    elif nivel == 2:
        title = 'ERRO: Falha na Chamada da API'
        msg = f"Erro na chamada da API {url} (Status Code diferente de 200).\n{now}\n{erro}"
    elif nivel == 3:
        title = 'ERRO CRÍTICO: API Indisponível'
        msg = f"A API {url} parece estar indisponível ou houve um erro de conexão.\n{now}\n{erro}"
    else:
        print("Nível", nivel, "não disponível para alerta da API!")
        return  

    if title:
        notification.notify(
            title=title,
            message=msg,
            app_name='alerta_api',
            timeout=10
        )
    else:
        print(f"Alerta da API para {url}: Nível inválido.") 

In [4]:
def obter_dados_da_api(url, token):
    try:
        response = requests.get(url, token)
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a API: {e}")
        return None


In [5]:
def criar_dataframe(data):
    if data is None:
        return None
    
    paises = []
    for pais in data['data']:
        pais_info = {
            'Nome Comum': pais['name'],
            'Nome Oficial': pais['full_name'],
            'Código Alpha-2': pais['iso2'],
            'Capital': pais['capital']
        }
        paises.append(pais_info)
    
    return pd.DataFrame(paises)

In [6]:
def criar_dataframe_populacao(data):
    if data is None:
        return None
    
    paises = []
    for pais in data['data']:
        pais_info = {
            'Nome Comum': pais['name'],
            'População': pais['covid19']['total_case'],
            'Área': pais['size'],
            'Continente': pais['continent']
        }
        paises.append(pais_info)
    
    return pd.DataFrame(paises)


In [7]:
def criar_dataframe_membro_da_onu(data):
    if data is None:
        return None
    
    paises = []
    for pais in data['data']:
        pais_info = {
            'Nome Comum': pais.get('name'),
                        'Nome Oficial': pais.get('full_name'),
                        'Código Alpha-2': pais.get('iso2'),
                        'Capital': pais.get('capital'),
                        'População': pais.get('population'),
                        'Área': pais.get('size'),
                        'Continente': pais.get('continent'),
                        'Independente': pais.get('independent'),
                        'Membro da ONU': pais.get('unMember'),
                        'Status': pais.get('status'), 
        }
        paises.append(pais_info)
    
    return pd.DataFrame(paises)

In [8]:
df_paises = criar_dataframe(data)
df_populacao = criar_dataframe_populacao(data)
df_onu = criar_dataframe_membro_da_onu(data)

In [9]:
current_dir = os.getcwd()
db_path = os.path.join(current_dir, 'BD', 'dados_analise.db')
os.makedirs(os.path.dirname(db_path), exist_ok=True) #cria a pasta DB se ela não existir
conn = sqlite3.connect(db_path)
conn.close()

In [10]:
conn = sqlite3.connect(db_path)

nome_tabela_paises = 'paises'
df_paises.to_sql(nome_tabela_paises, conn, if_exists='replace', index=False)
nome_tabela_populacao = 'populacao'
df_populacao.to_sql(nome_tabela_populacao, conn, if_exists='replace', index=False)
nome_tabela_onu = 'membros_onu'
df_onu.to_sql(nome_tabela_onu, conn, if_exists='replace', index=False)
conn.close()

In [11]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table in tables:
    print(table[0])
conn.close()

paises
populacao
membros_onu


In [12]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

tabelas = ['paises', 'populacao', 'membros_onu']

for tabela in tabelas:
    print(f"\nColunas da tabela '{tabela}':")
    cursor.execute(f"PRAGMA table_info({tabela})")
    colunas_info = cursor.fetchall()

    if colunas_info:
        for coluna in colunas_info:
            print(f"- {coluna[1]}")
    else:
        print(f"A tabela '{tabela}' não foi encontrada.")
conn.close()


Colunas da tabela 'paises':
- Nome Comum
- Nome Oficial
- Código Alpha-2
- Capital

Colunas da tabela 'populacao':
- Nome Comum
- População
- Área
- Continente

Colunas da tabela 'membros_onu':
- Nome Comum
- Nome Oficial
- Código Alpha-2
- Capital
- População
- Área
- Continente
- Independente
- Membro da ONU
- Status


In [13]:
df_paises.head()

,Nome Comum,Nome Oficial,Código Alpha-2,Capital
0,Afghanistan,Islamic Republic of Afghanistan,AF,Kabul
1,Albania,Republic of Albania | Shqipëria,AL,Tirana
2,Algeria,Democratic Republic of Algeria,DZ,Algiers
3,Andorra,Principality of Andorra,AD,Andorra la Vella
4,Angola,Republic of Angola,AO,Luanda


In [14]:
df_paises.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Nome Comum      186 non-null    object
 1   Nome Oficial    184 non-null    object
 2   Código Alpha-2  186 non-null    object
 3   Capital         186 non-null    object
dtypes: object(4)
memory usage: 5.9+ KB


In [15]:
df_paises.shape

(186, 4)

In [16]:
df_paises.columns

Index(['Nome Comum', 'Nome Oficial', 'Código Alpha-2', 'Capital'], dtype='object')

In [17]:
df_populacao.head()

,Nome Comum,População,Área,Continente
0,Afghanistan,"46,498","652,000 km²",Asia
1,Albania,"37,625","28,748 km²",Europe
2,Algeria,"83,199","2,381,741 km²",Africa
3,Andorra,"6,712",468 km²,Europe
4,Angola,"15,139","1,246,620 km²",Africa


In [18]:
df_populacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Nome Comum  186 non-null    object
 1   População   186 non-null    object
 2   Área        186 non-null    object
 3   Continente  184 non-null    object
dtypes: object(4)
memory usage: 5.9+ KB


In [19]:
df_populacao.shape

(186, 4)

In [20]:
df_populacao.columns

Index(['Nome Comum', 'População', 'Área', 'Continente'], dtype='object')

In [21]:
df_onu.head()

,Nome Comum,Nome Oficial,Código Alpha-2,Capital,População,Área,Continente,Independente,Membro da ONU,Status
0,Afghanistan,Islamic Republic of Afghanistan,AF,Kabul,"39,306,195","652,000 km²",Asia,None,None,None
1,Albania,Republic of Albania | Shqipëria,AL,Tirana,"2,876,490","28,748 km²",Europe,None,None,None
2,Algeria,Democratic Republic of Algeria,DZ,Algiers,"44,190,030","2,381,741 km²",Africa,None,None,None
3,Andorra,Principality of Andorra,AD,Andorra la Vella,"77,317",468 km²,Europe,None,None,None
4,Angola,Republic of Angola,AO,Luanda,"33,312,843","1,246,620 km²",Africa,None,None,None


In [22]:
df_onu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Nome Comum      186 non-null    object
 1   Nome Oficial    184 non-null    object
 2   Código Alpha-2  186 non-null    object
 3   Capital         186 non-null    object
 4   População       186 non-null    object
 5   Área            186 non-null    object
 6   Continente      184 non-null    object
 7   Independente    0 non-null      object
 8   Membro da ONU   0 non-null      object
 9   Status          0 non-null      object
dtypes: object(10)
memory usage: 14.7+ KB


In [23]:
df_onu.shape

(186, 10)

In [24]:
df_onu.columns

Index(['Nome Comum', 'Nome Oficial', 'Código Alpha-2', 'Capital', 'População',
       'Área', 'Continente', 'Independente', 'Membro da ONU', 'Status'],
      dtype='object')

## Problema com Dados de Filiados da ONU e Busca na API

Precisei buscar informações mais precisas e atualizadas diretamente na **API oficial da ONU**.


In [25]:
url = 'https://population.un.org/dataportalapi/api/v1/locais'

if response.status_code == 200:
    membros_onu = response.json()
else:
    print(f"Erro na requisição: {response.status_code}")
    print(response.text)

df_membros = pd.DataFrame(membros_onu['data'])
df_membros.columns


Index(['name', 'full_name', 'capital', 'iso2', 'iso3', 'covid19',
       'current_president', 'currency', 'phone_code', 'continent',
       'description', 'size', 'independence_date', 'population', 'href'],
      dtype='object')

In [26]:
url = 'https://population.un.org/dataportalapi/api/v1/locais'
response = requests.get(url)

if response.status_code == 200:
    membros_onu = response.json()
    df_membros = pd.DataFrame(membros_onu['data'])
    df_membros = df_membros[['name']]  


## A NAME corresponde aos dados que nos falta:

**`name`**

In [27]:
coluna_nome = df_membros['name']
coluna_nome


0      Afghanistan
1          Albania
2          Algeria
3          Andorra
4           Angola
          ...     
181      Venezuela
182        Vietnam
183          Yemen
184         Zambia
185       Zimbabwe
Name: name, Length: 186, dtype: object

In [28]:
nome_tabela = "onu"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute(f"PRAGMA table_info({nome_tabela});")
columns_info = cursor.fetchall()
for column in columns_info:
    print(column[1])

In [29]:
nome_nova_coluna = 'paises_membros_da_ONU'
df_onu[nome_nova_coluna] = coluna_nome

In [30]:
df_onu.head()

,Nome Comum,Nome Oficial,Código Alpha-2,Capital,População,Área,Continente,Independente,Membro da ONU,Status,paises_membros_da_ONU
0,Afghanistan,Islamic Republic of Afghanistan,AF,Kabul,"39,306,195","652,000 km²",Asia,None,None,None,Afghanistan
1,Albania,Republic of Albania | Shqipëria,AL,Tirana,"2,876,490","28,748 km²",Europe,None,None,None,Albania
2,Algeria,Democratic Republic of Algeria,DZ,Algiers,"44,190,030","2,381,741 km²",Africa,None,None,None,Algeria
3,Andorra,Principality of Andorra,AD,Andorra la Vella,"77,317",468 km²,Europe,None,None,None,Andorra
4,Angola,Republic of Angola,AO,Luanda,"33,312,843","1,246,620 km²",Africa,None,None,None,Angola


In [31]:
df_onu.columns

Index(['Nome Comum', 'Nome Oficial', 'Código Alpha-2', 'Capital', 'População',
       'Área', 'Continente', 'Independente', 'Membro da ONU', 'Status',
       'paises_membros_da_ONU'],
      dtype='object')

In [32]:
db_path = 'BD/dados_analise.db'  
nome_coluna_paises_membros = 'paises_membros_da_ONU'  # Nome de uma coluna (não usada diretamente)
nome_coluna_nome_comum = 'Nome Comum'  # Nome da coluna que contém os nomes dos países
nome_coluna_membro_onu = 'Membro da ONU'  # Nome da nova coluna a ser criada

conn = sqlite3.connect(db_path)  # Conecta ao banco de dados
df_onu = pd.read_sql_query(f"SELECT * FROM {nome_tabela_onu}", conn)  # Lê a tabela "onu" para um DataFrame

nome_nova_coluna = 'paises_membros_da_ONU'
df_onu[nome_nova_coluna] = coluna_nome  # Adiciona uma coluna com os nomes dos países membros

def verificar_membro(row):
    nome_comum = row[nome_coluna_nome_comum]  # Pega o nome do país na linha atual
    if nome_coluna_paises_membros in row:  # Verifica se a coluna de membros existe
        string_paises_membros = row[nome_coluna_paises_membros]  # Pega a lista de membros
        if isinstance(string_paises_membros, str) and nome_comum in string_paises_membros:
            return 'YES'  # Se o país está na lista, retorna "YES"
        else:
            return 'NO'  # Caso contrário, retorna "NO"
    else:
        return 'Coluna "paises_membros_da_ONU" não encontrada na linha'
    
df_onu[nome_coluna_membro_onu] = df_onu.apply(verificar_membro, axis=1)

nome_coluna_para_excluir = 'paises_membros_da_ONU'
df_onu = df_onu.drop(columns=[nome_coluna_para_excluir], axis=1, errors='ignore')

conn.close()  # Fecha a conexão com o banco de dados
df_onu.head()  # Mostra as primeiras linhas do DataFrame

,Nome Comum,Nome Oficial,Código Alpha-2,Capital,População,Área,Continente,Independente,Membro da ONU,Status
0,Afghanistan,Islamic Republic of Afghanistan,AF,Kabul,"39,306,195","652,000 km²",Asia,None,YES,None
1,Albania,Republic of Albania | Shqipëria,AL,Tirana,"2,876,490","28,748 km²",Europe,None,YES,None
2,Algeria,Democratic Republic of Algeria,DZ,Algiers,"44,190,030","2,381,741 km²",Africa,None,YES,None
3,Andorra,Principality of Andorra,AD,Andorra la Vella,"77,317",468 km²,Europe,None,YES,None
4,Angola,Republic of Angola,AO,Luanda,"33,312,843","1,246,620 km²",Africa,None,YES,None


In [33]:
db_path = 'BD/dados_analise.db'  
nome_tabela_onu = "onu"
nome_coluna_paises_membros = 'paises_membros_da_ONU'
nome_coluna_nome_comum = 'Nome Comum'
nome_coluna_membro_onu = 'Membro da ONU'

conn = sqlite3.connect(db_path)
nome_coluna_para_excluir = 'Independente'
df_onu = df_onu.drop(columns=[nome_coluna_para_excluir], axis=1, errors='ignore')
conn.close()


df_onu.head() 



,Nome Comum,Nome Oficial,Código Alpha-2,Capital,População,Área,Continente,Membro da ONU,Status
0,Afghanistan,Islamic Republic of Afghanistan,AF,Kabul,"39,306,195","652,000 km²",Asia,YES,None
1,Albania,Republic of Albania | Shqipëria,AL,Tirana,"2,876,490","28,748 km²",Europe,YES,None
2,Algeria,Democratic Republic of Algeria,DZ,Algiers,"44,190,030","2,381,741 km²",Africa,YES,None
3,Andorra,Principality of Andorra,AD,Andorra la Vella,"77,317",468 km²,Europe,YES,None
4,Angola,Republic of Angola,AO,Luanda,"33,312,843","1,246,620 km²",Africa,YES,None


In [34]:
db_path = 'BD/dados_analise.db'  
nome_coluna_paises_membros = 'paises_membros_da_ONU'
nome_coluna_nome_comum = 'Nome Comum'
nome_coluna_membro_onu = 'Membro da ONU'

conn = sqlite3.connect(db_path)
nome_coluna_para_excluir = 'Status'
df_onu = df_onu.drop(columns=[nome_coluna_para_excluir], axis=1, errors='ignore')
conn.close()


df_onu.head() 


,Nome Comum,Nome Oficial,Código Alpha-2,Capital,População,Área,Continente,Membro da ONU
0,Afghanistan,Islamic Republic of Afghanistan,AF,Kabul,"39,306,195","652,000 km²",Asia,YES
1,Albania,Republic of Albania | Shqipëria,AL,Tirana,"2,876,490","28,748 km²",Europe,YES
2,Algeria,Democratic Republic of Algeria,DZ,Algiers,"44,190,030","2,381,741 km²",Africa,YES
3,Andorra,Principality of Andorra,AD,Andorra la Vella,"77,317",468 km²,Europe,YES
4,Angola,Republic of Angola,AO,Luanda,"33,312,843","1,246,620 km²",Africa,YES


In [35]:
df_onu['Membro da ONU'].value_counts()

YES    186
Name: Membro da ONU, dtype: int64

In [36]:
total_yes = (df_onu['Membro da ONU'] == 'YES').sum()
total_no = (df_onu['Membro da ONU'] == 'NO').sum()
total_nulos = df_onu['Membro da ONU'].isna().sum()  # Verifica valores nulos

print(f"\nTotal 'YES': {total_yes}")
print(f"Total 'NO': {total_no}")
print(f"Total nulos: {total_nulos}")


Total 'YES': 186
Total 'NO': 0
Total nulos: 0
